# Working with Chandra FITS tables

## Authors
Lia Corrales, Kris Stern, Lúthien  Liu, Zihao Chen, Saima Siddiqui

## Learning Goals
* Download a Chandra FITS table file from a URL 
* Open a Chandra FITS table file and view table contents
* Make a 2D histogram with the event list data
* Close the FITS file after use

## Keywords
FITS, file input/output, table, numpy, matplotlib, histogram


## Summary

Chandra image data is stored in FITS files, frequently referred to as "event lists". Any time a photon interacts with the detector, the position, time, and energy of the photon (referred to as an "event") is stored. Thus Chandra event lists are stored as table data, where the position and energy information is stored in the columns and each row corresponds to a separate photon interaction.

In this tutorial, we will use `astropy.utils.data` to download a Chandra FITS file, then use `astropy.io.fits` and `astropy.table` to open the file. Lastly, we will use `matplotlib` to visualize the Chandra event list as a histogram, effectively producing an X-ray image of the sky.

In [ ]:
import numpy as np
from astropy.io import fits
from astropy.table import Table
from matplotlib.colors import LogNorm

# Set up matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

The following line is needed to download the example FITS files used in this tutorial.

In [ ]:
from astropy.utils.data import download_file

FITS files often contain large amounts of multi-dimensional data and tables.  

In this particular example, we'll open a FITS file from a Chandra observation of the Galactic Center.  The file contains a list of events with x and y coordinates, energy, and various other pieces of information.

In [ ]:
event_filename = download_file('http://data.astropy.org/tutorials/FITS-tables/chandra_events.fits', 
                               cache=True)

## Opening the FITS file and viewing table contents

Since the file is big, let's open it with `memmap=True` to prevent RAM storage issues.

In [ ]:
hdu_list = fits.open(event_filename, memmap=True)

In [ ]:
hdu_list.info()

In this case, we're interested in reading EVENTS, which contains information about each X-ray photon that hit the detector.

To find out what information the table contains, let's print the column names.

In [ ]:
print(hdu_list[1].columns)

Now we'll take this data and convert it into an [astropy table](http://docs.astropy.org/en/stable/table/). While it's possible to access FITS tables directly from the ``.data`` attribute, using [Table](http://docs.astropy.org/en/stable/api/astropy.table.Table.html#astropy.table.Table) tends to make a variety of common tasks more convenient.

In [ ]:
evt_data = Table(hdu_list[1].data)

For example, a preview of the table is easily viewed by simply running a cell with the table as the last line:

In [ ]:
evt_data

We can extract data from the table by referencing the column name. Let's try making a histogram for the energy of each photon, which will give us a sense for the spectrum (folded with the detector's efficiency).

In [ ]:
energy_hist = plt.hist(evt_data['energy'], bins='auto')
plt.xlabel('Energy (eV)')
plt.ylabel('Number of photon events')

## Making a 2D histogram with some table data

Next we'll make an image by binning the x and y coordinates of the events into a 2D histogram. 

A one dimensional histogram, as shown above, shows the number of events within each bin corresponding to one axis of information. In the plot above, we chose histogram bins in the energy, shown on the x-axis.

A two dimensional histogram finds the number of events binned according to two dimensions. To make an image, we will bin the number of events by x and y position on the sky.

This particular observation spans five CCD chips.  First, we determine the events that only fell on the main (ACIS-I) chips, which have number ids 0, 1, 2, and 3. We can do this by creating an array of True and False values (`ii` below) to filter out events that only fall on those chips.

In [ ]:
ii = np.isin(evt_data['ccd_id'], [0, 1, 2, 3])

### Method 1: Use hist2d with a log-normal color scheme

We can make a 2D histogram plot directly with the function `matplotlib.pyplot.hist2d`, as shown below.

In this example, we choose the `matplotlib` color map named "viridis", and we choose to distribute the colors logarithmically using `matplotlib.colors.LogNorm()`.

To see what colormaps are available with `matplotlib`, see http://wiki.scipy.org/Cookbook/Matplotlib/Show_colormaps


In [ ]:
NBINS = (100,100)
img_zero_mpl = plt.hist2d(evt_data['x'][ii], evt_data['y'][ii], NBINS, 
                          cmap='viridis', norm=LogNorm())

# Show the color bar scale next to the plot. The color corresponds to number 
# of photon events (counts) in each pixel.
cbar = plt.colorbar(label='Counts')

plt.xlabel('x')
plt.ylabel('y')

### Method 2: Use numpy to make a 2D histogram and imshow to display it

When we plot with `matplotlib.pyplot.hist2d`, it forces the plot into the default figure size, which could cause your image to appear stretched. 

By using `matplotlib.pyplot.imshow`, we can avoid stretching the image. To do that, we need to make a 2D array containing the number of counts per pixel bin using `numpy.histogram2d`, as shown below.

In [ ]:
NBINS = (100,100)

img_zero, xedges, yedges = np.histogram2d(evt_data['y'][ii], evt_data['x'][ii], NBINS)

# This array describes how to map the position of the 2D array containing the image
# to the x and y positions on the sky
extent = [xedges[0], xedges[-1], yedges[0], yedges[-1]]

plt.imshow(img_zero, extent=extent, interpolation='nearest', 
           cmap='gist_yarg', origin='lower', norm=LogNorm())

plt.xlabel('x')
plt.ylabel('y')

## Close the FITS file

When you're done using a FITS file, it's often a good idea to close it.  That way you can be sure it won't continue using up excess memory or file handles on your computer.  (This happens automatically when you close Python, but you never know how long that might be...)

In [ ]:
hdu_list.close()

## Exercises

Make a scatter plot of the same data you histogrammed above.  The [plt.scatter](http://matplotlib.org/api/pyplot_api.html#matplotlib.pyplot.scatter) function is your friend for this.  What are the pros and cons of doing it this way?

In [ ]:
energy_scatter = plt.scatter(evt_data['x'][ii], evt_data['y'][ii], s=1, alpha=0.1, color='b')

Try the same with the [plt.hexbin](http://matplotlib.org/api/pyplot_api.html#matplotlib.pyplot.hexbin) plotting function. Which do you think looks better for this kind of data?

In [ ]:
energy_hex = plt.hexbin(evt_data['x'][ii], evt_data['y'][ii], norm=LogNorm())

Choose an energy range to make a slice of the FITS table, then plot it. How does the image change with different energy ranges?

In [ ]:
NBINS = (100,100)
ii = (evt_data['energy'] > 3500) & (evt_data['energy'] < 5000)
img_zero_mpl = plt.hist2d(evt_data['x'][ii], evt_data['y'][ii], NBINS,
                          cmap='viridis', norm=LogNorm())

cbar = plt.colorbar(ticks=[1.0,3.0,6.0])
cbar.ax.set_yticklabels(['1','3','6'])

plt.xlabel('x')
plt.ylabel('y')